## 3-ways of LLM connectivity
* Chat: string input & string output
* ChatCompletion: list of conversation as input & string output
    * Chat & ChatCompletion are stateless
        * means we need to provide information to LLM evey time we communicate with it

        * There are four ways by wich we can provide pevious conversation's history to LLM
            * Providing complete history of previous conversations
            * Fixing wondow (no of latest conversations)
            * Fixing tokens limit
            * Summerization
* Assistant: 
    * This is stateful (conversation's history is retained)
    * It requires to import 3 classes
        * Assistant: For example when we open ChatGPT, the prompt says "How can I help you today?"; it is an assistant
            * Requires three parameters: name, instructions and which LLM to use
        * Thread: series of input(prompts) and outputs(responses) realted to the Assistant
        * Run: Processing of inputs & outputs and selecting the appropriate Assistant.
            * Takes 2 arguments, Assistant (id) & Thread(id)
            * Has 3 status: queued, in-process & completed (status of the task being performed)

In [ ]:
%pip install -r requirements.txt

In [2]:
from dotenv import find_dotenv, load_dotenv

_ : bool = load_dotenv(find_dotenv())

In [3]:
from openai import OpenAI
client : OpenAI = OpenAI()

# code_interpreter

### Step-1: Create Assistant

In [ ]:
from openai.types.beta.assistant import Assistant

assistant : Assistant = client.beta.assistants.create(
    name="Math tutor",
    instructions="Tou are a personal math tutor. Write and run code to answer math questions",
    model="gpt-3.5-turbo-1106",
    tools=[{"type":"code_intrepreter"}]     # can be a "code_interpreter" or a "file_retriever" or "function_calling"
)

### Step-2: Create a Thread
* Takes no parameter
* Inside of an Assistant there may be multiple threads 
* Inside of a thread, there are multiple messages (between the user & the assistant)
    * We create messages, put these messages into threads and link these threads to the assistants

In [ ]:
from openai.types.beta.thread import Thread

thread : Thread = client.beta.threads.create()
print(thread)

### Step-3: Add a message to a Thread
* Takes 3 parameters
    * Thread ID
    * Role of the user
    * Content of the message

In [ ]:
from openai.types.beta.threads.thread_message import ThreadMessage

message : ThreadMessage = client.beta.threads.messages.create(
    thread_id = thread.id,     # from previously created thread
    role = "user",
    content = "Solve this problem: 3x + 11 = 14"
)

### Step-4: Run the Assistant
* Takes 2 parameter
    * Thread ID
    * Assistant ID

In [ ]:
from openai.types.beta.threads.run import Run

run : Run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Please address the user as Jane Doe. The user has a premium account"
)

### Check the Run Status

In [ ]:
run : Run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id
)

print(run)

### Step-5: Dispaly the Assistant's Response
* To display the Assistant's response we need to retrieve the messages from the thread

In [ ]:
# To retrieve the run
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [ ]:
# Now we need to retrieve the messages from the run
# messages : list[ThreadMessage] = client.beta.threads.messages.retrieve(
#     thread_id = thread.id
# )

messages : list[ThreadMessage] = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [ ]:
# To display all the messages sent beteen the user & assistant

for message in reversed(messages.data):  # reversed in order to get the oldest messages printrd out first and then the latest
    print(message.role + ": " + message.content[0].text.value)